In [1]:
import pandas as pd

In [2]:
speakers = pd.read_csv("data/speakers_all.csv", usecols=["native_language", "filename", "file_missing?"])

In [3]:
speakers.head()

,filename,native_language,file_missing?
0,balanta,balanta,True
1,cameroon,cameroon,True
2,fulfulde,fulfulde,True
3,haitian,haitian,True
4,haitian,haitian,True


In [4]:
speakers.dtypes

filename           object
native_language    object
file_missing?        bool
dtype: object

In [5]:
import sys, os, time
from subprocess import Popen, list2cmdline
import subprocess

In [6]:
cmd = "./featex -i ./data/converted/{}.raw " \
    "please call stella ask her to bring these " \
    "things with her from the store six spoons " \
    "of fresh snow peas five thick slabs of blue " \
    "cheese and maybe a snack for her brother " \
    "bob we also need a small plastic snake and " \
    "a big toy frog for the kids she can scoop " \
    "these things into three red bags and we will " \
    "go meet her wednesday at the train station"

In [7]:
data_file = "data/dataset.csv"

In [8]:
# Initialize the csv file
columns = ['native_language']
columns.extend(range(25))
df = pd.DataFrame([], columns=columns)
df.to_csv(data_file, index=False)

In [9]:
def append_to_data(native, data):
    record = [[native]]
    ps = data.decode().split('\n')
    wednesday = ps[202: 208]
    end = float(ps[208].split()[-1])

    for p in wednesday:
        for num in p.split()[1:]:
            record[0].append(float(num))
    record[0].append(end)
    df = pd.DataFrame(record, columns=columns)
    df.to_csv(data_file, mode='a', header=False, index=False)

In [10]:
def cpu_count():
    ''' Returns the number of CPUs in the system
    '''
    num = 1
    if sys.platform == 'win32':
        try:
            num = int(os.environ['NUMBER_OF_PROCESSORS'])
        except (ValueError, KeyError):
            pass
    elif sys.platform == 'darwin':
        try:
            num = int(os.popen('sysctl -n hw.ncpu').read())
        except ValueError:
            pass
    else:
        try:
            num = os.sysconf('SC_NPROCESSORS_ONLN')
        except (ValueError, OSError, AttributeError):
            pass

    return num

In [30]:
def exec_commands(cmds):
    ''' Exec commands in parallel in multiple process 
    (as much as we have CPU)
    '''
    if not cmds: return # empty list

    def done(p):
        return p.poll() is not None
    def success(p):
        return p.returncode == 0
    def fail():
        sys.exit(1)
    
    max_task = cpu_count()
    processes = []
    while True:
        while cmds and len(processes) < max_task:
            task = cmds.pop()
            processes.append((
                Popen(
                    task[0],
                    env={"LD_LIBRARY_PATH" : "/usr/local/lib"},
                    stdout=subprocess.PIPE
                )
                ,task[1]
            ))

        for p, native in processes:
            if done(p):
                processes.remove((p, native))
                if success(p):
                    append_to_data(native, p.communicate()[0])

        if not processes and not cmds:
            break
        else:
            time.sleep(0.05)

In [31]:
cmds = []

for index, row in speakers.iterrows():
    if row["file_missing?"]:
        continue
    cmds.append((cmd.format(str(row["filename"])).split(), row["native_language"]))
cmds = cmds[:-393]

In [32]:
exec_commands(cmds)